# 🎨 딥러닝 3일차 복습: DCGAN & CGAN 완전 정복

이번 복습에서는 3일차에 학습한 **DCGAN**과 **CGAN**의 핵심 개념들을 다시 정리하고, 실습 문제를 통해 이해도를 점검합니다.

## 📚 학습 목표
- DCGAN의 5가지 설계 원칙과 아키텍처 이해
- Transposed Convolution의 동작 원리 파악
- Conditional GAN의 조건 정보 결합 방식 이해
- Embedding vs One-Hot Encoding 차이점 명확히 구분
- BatchNorm과 InstanceNorm의 역할과 차이점 이해
- PyTorch로 DCGAN/CGAN 구현 능력 향상

---

## ⚙️ 환경 설정

In [ ]:
# 필요한 라이브러리 임포트
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt

# 한글 폰트 설정
plt.rc('font', family='NanumGothic')
plt.rcParams['axes.unicode_minus'] = False

# 시드 고정
torch.manual_seed(42)
np.random.seed(42)

# GPU 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 디바이스: {device}")
if torch.cuda.is_available():
    print(f"GPU 이름: {torch.cuda.get_device_name(0)}")
    print(f"GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

---
# Part 1: 객관식 퀴즈 🧠

먼저 DCGAN과 CGAN의 핵심 내용을 객관식 문제로 점검해봅시다.

## 퀴즈 1: DCGAN의 설계 원칙

**문제:** DCGAN에서 Pooling Layer를 제거하고 대신 사용하는 기법은 무엇인가요?

1. Max Pooling
2. Average Pooling
3. Strided Convolution
4. Dilated Convolution

<details>
<summary><b>정답 및 풀이 보기</b></summary>

**정답: 3번 (Strided Convolution)**

**풀이:**
- DCGAN의 첫 번째 설계 원칙은 **"Pooling Layer 제거"**입니다.
- 대신 **Strided Convolution**을 사용하여 다운샘플링을 수행합니다.
- 장점:
  - 학습 가능한 다운샘플링 (파라미터가 있음)
  - 정보 손실 최소화 (MaxPooling은 3/4 정보 버림)
  - Gradient 흐름 개선

**예시:**
```python
# Pooling 방식 (X)
nn.MaxPool2d(2, 2)

# DCGAN 방식 (O)
nn.Conv2d(64, 128, 4, stride=2, padding=1)
```

</details>

## 퀴즈 2: BatchNormalization 적용 위치

**문제:** DCGAN에서 BatchNormalization을 적용하지 **않는** 층은? (2개 선택)

1. Generator의 첫 번째 은닉층
2. Generator의 출력층
3. Discriminator의 입력층
4. Discriminator의 은닉층

<details>
<summary><b>정답 및 풀이 보기</b></summary>

**정답: 2번, 3번**

**풀이:**
- **Generator 출력층 (2번)**: Tanh 활성화로 [-1, 1] 범위를 유지해야 하므로 BatchNorm 적용 안 함
- **Discriminator 입력층 (3번)**: 원본 이미지 분포를 보존해야 하므로 BatchNorm 적용 안 함

**BatchNorm 적용 규칙:**
```
Generator:
Input → Dense → [BatchNorm] → ReLU
     → ConvTranspose → [BatchNorm] → ReLU
     → ConvTranspose → [BatchNorm] → ReLU
     → ConvTranspose → Tanh (BatchNorm X)

Discriminator:
Input → Conv → LeakyReLU (BatchNorm X)
     → Conv → [BatchNorm] → LeakyReLU
     → Conv → [BatchNorm] → LeakyReLU
     → Dense → Sigmoid
```

</details>

## 퀴즈 3: Transposed Convolution

**문제:** Transposed Convolution에서 입력 크기가 4×4, kernel_size=4, stride=2, padding='same'일 때 출력 크기는?

1. 2×2
2. 4×4
3. 8×8
4. 16×16

<details>
<summary><b>정답 및 풀이 보기</b></summary>

**정답: 3번 (8×8)**

**풀이:**
- `padding='same'`이고 `stride=2`일 때: **출력 크기 = 입력 크기 × stride**
- 계산: 4 × 2 = 8

**일반 공식:**
```
padding='same'일 때:
output_size = input_size × stride

예시:
4×4  → 8×8   (stride=2)
8×8  → 16×16 (stride=2)
16×16 → 32×32 (stride=2)
```

**PyTorch 코드:**
```python
nn.ConvTranspose2d(256, 128, 4, stride=2, padding=1)
# 4×4 입력 → 8×8 출력
```

</details>

## 퀴즈 4: DCGAN의 활성화 함수

**문제:** DCGAN에서 사용하는 활성화 함수 조합으로 올바른 것은?

1. Generator 은닉층: Sigmoid / Discriminator 은닉층: ReLU
2. Generator 은닉층: ReLU / Discriminator 은닉층: LeakyReLU
3. Generator 은닉층: LeakyReLU / Discriminator 은닉층: ReLU
4. Generator 은닉층: Tanh / Discriminator 은닉층: Sigmoid

<details>
<summary><b>정답 및 풀이 보기</b></summary>

**정답: 2번**

**풀이:**
- **Generator 은닉층**: ReLU (Gradient 흐름 원활)
- **Generator 출력층**: Tanh (출력 범위 [-1, 1])
- **Discriminator 은닉층**: LeakyReLU(0.2) (Dying ReLU 방지)
- **Discriminator 출력층**: Sigmoid 또는 없음 (BCEWithLogits 사용 시)

**LeakyReLU를 사용하는 이유:**
```
ReLU: f(x) = max(0, x)
→ x < 0일 때 gradient = 0 (뉴런 사망)

LeakyReLU: f(x) = max(0.2x, x)
→ x < 0일 때도 gradient = 0.2 (학습 가능)
```

</details>

## 퀴즈 5: Conditional GAN의 핵심

**문제:** Conditional GAN (CGAN)이 Vanilla GAN과 다른 핵심적인 차이는?

1. Discriminator만 조건 정보를 받음
2. Generator만 조건 정보를 받음
3. Generator와 Discriminator 모두 조건 정보를 받음
4. 조건 정보는 Loss 함수에만 반영됨

<details>
<summary><b>정답 및 풀이 보기</b></summary>

**정답: 3번**

**풀이:**
- CGAN은 **Generator와 Discriminator 모두** 조건 정보(y)를 입력으로 받습니다.

**구조 비교:**
```
Vanilla GAN:
G(z) → 이미지
D(이미지) → 진짜/가짜

Conditional GAN:
G(z, y) → 조건에 맞는 이미지
D(이미지, y) → 이미지와 조건의 페어링 판별
```

**예시 (Fashion MNIST):**
- Generator: noise + "Sneaker" 레이블 → 스니커즈 이미지 생성
- Discriminator: 스니커즈 이미지 + "Sneaker" 레이블 → 진짜 판별
- Discriminator: 스니커즈 이미지 + "T-shirt" 레이블 → 가짜 판별 (페어링 안 맞음!)

</details>

## 퀴즈 6: Embedding vs One-Hot Encoding

**문제:** CGAN에서 클래스 레이블을 처리할 때, Embedding 방식이 One-Hot Encoding보다 유리한 이유는?

1. 구현이 더 간단해서
2. 파라미터 수가 더 적어서
3. 클래스 간 유사도를 학습할 수 있어서
4. 학습 속도가 더 빨라서

<details>
<summary><b>정답 및 풀이 보기</b></summary>

**정답: 3번**

**풀이:**

**One-Hot Encoding:**
```python
# 10개 클래스 → 10차원 벡터
Class 0 (T-shirt): [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Class 1 (Trouser): [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
Class 7 (Sneaker): [0, 0, 0, 0, 0, 0, 0, 1, 0, 0]

문제점:
- 모든 클래스가 동일한 거리 (직교 벡터)
- 유사한 클래스(T-shirt vs Shirt)도 먼 관계
- 클래스 수가 많으면 비효율 (1000개 → 1000차원)
```

**Embedding:**
```python
# 10개 클래스 → 50차원 학습 가능한 벡터
nn.Embedding(num_classes=10, embedding_dim=50)

학습 후:
Class 0 (T-shirt):  [0.2, -0.5, 0.8, ..., 0.1]
Class 6 (Shirt):    [0.3, -0.4, 0.7, ..., 0.2]  ← T-shirt와 유사!
Class 7 (Sneaker):  [-0.6, 0.7, 0.2, ..., -0.3] ← T-shirt와 다름

장점:
✅ 클래스 간 유사도 학습 (의미 벡터)
✅ 파라미터 효율적 (10×50 = 500 vs 1000×1000)
✅ 차원 조절 가능 (50, 100, 200 등)
```

**시각화 예시:**
```
t-SNE로 임베딩 벡터를 2D 투영하면:
- 상의류 (T-shirt, Shirt, Pullover) → 가까이 모임
- 하의류 (Trouser) → 따로 위치
- 신발류 (Sneaker, Sandal, Ankle boot) → 가까이 모임
```

</details>

---
# Part 2: 서술형 문제 (핵심 개념) 📝

## 문제 1: DCGAN의 5가지 설계 원칙

**문제:** DCGAN 논문에서 제시한 5가지 설계 원칙을 모두 나열하고, 각각이 어떤 문제를 해결하는지 설명하세요.

---

### 여러분의 답안:
```
[여기에 답안을 작성하세요]

1. 
2. 
3. 
4. 
5. 

```

<details>
<summary><b>모범 답안 보기</b></summary>

**DCGAN의 5가지 설계 원칙:**

| 원칙 | 내용 | 해결하는 문제 |
|------|------|-------------|
| **1. Strided Convolution** | Pooling Layer 제거, stride=2로 다운샘플링 | 학습 가능한 다운샘플링, 정보 손실 최소화 |
| **2. FC Layer 제거** | 중간층에서 Fully Connected Layer 제거 | 공간 구조 보존, 파라미터 효율성 |
| **3. Batch Normalization** | 모든 층에 BatchNorm 적용 (입력/출력층 제외) | Internal Covariate Shift 방지, 학습 안정화 |
| **4. 적절한 활성화 함수** | G: ReLU/Tanh, D: LeakyReLU(0.2) | Gradient 흐름 개선, Dying ReLU 방지 |
| **5. Adam Optimizer** | lr=0.0002, beta1=0.5, beta2=0.999 | GAN의 불안정성에 최적화된 설정 |

**각 원칙의 효과:**

1. **Strided Convolution**: MaxPooling은 정보의 3/4를 버리지만, Strided Conv는 모든 정보를 활용하여 학습 가능

2. **FC Layer 제거**: 
   - Vanilla GAN: 28×28 이미지를 784차원으로 flatten → 공간 정보 손실
   - DCGAN: Conv만 사용 → 2D 구조 유지

3. **Batch Normalization**:
   - 각 층의 입력 분포를 안정화 (평균 0, 분산 1)
   - Gradient vanishing/exploding 방지

4. **활성화 함수**:
   - LeakyReLU: 음수 영역에서도 gradient 유지 (α=0.2)
   - Tanh: 출력 범위 [-1, 1]로 이미지와 일치

5. **Adam (β1=0.5)**:
   - 기본값 0.9보다 낮춤 → 과거 gradient 의존도 감소
   - GAN의 불안정성에 더 빠르게 적응

</details>

## 문제 2: Transposed Convolution의 동작 원리

**문제:** Transposed Convolution이 일반 Convolution과 어떻게 다른지, 그리고 왜 "Upsampling"이 가능한지 설명하세요.

---

### 여러분의 답안:
```
[여기에 답안을 작성하세요]


```

<details>
<summary><b>모범 답안 보기</b></summary>

**Transposed Convolution의 핵심:**

**일반 Convolution (다운샘플링):**
```
입력: 4×4
커널: 3×3, stride=2
출력: 2×2

┌───┬───┬───┬───┐      ┌───┬───┐
│ a │ b │ c │ d │      │ * │ * │
├───┼───┼───┼───┤  →   ├───┼───┤
│ e │ f │ g │ h │      │ * │ * │
├───┼───┼───┼───┤      └───┴───┘
│ i │ j │ k │ l │
├───┼───┼───┼───┤
│ m │ n │ o │ p │
└───┴───┴───┴───┘
```

**Transposed Convolution (업샘플링):**
```
입력: 2×2
커널: 3×3, stride=2
출력: 4×4

┌───┬───┐      ┌───┬───┬───┬───┐
│ a │ b │      │ * │ * │ * │ * │
├───┼───┤  →   ├───┼───┼───┼───┤
│ c │ d │      │ * │ * │ * │ * │
└───┴───┘      ├───┼───┼───┼───┤
               │ * │ * │ * │ * │
               ├───┼───┼───┼───┤
               │ * │ * │ * │ * │
               └───┴───┴───┴───┘
```

**동작 원리:**
1. 입력의 각 픽셀이 커널 크기만큼의 출력 영역에 기여
2. stride만큼 띄워서 배치하고, 겹치는 부분은 더함
3. 학습 가능한 커널로 "어떻게" 확대할지 학습

**왜 Upsampling이 가능한가?**
- 일반 Convolution의 역연산으로 볼 수 있음
- 하지만 정확한 역연산은 아니고, "학습 가능한 업샘플링"
- Bilinear Upsampling과 달리 **학습 가능**한 점이 핵심

**주의사항: Checkerboard Artifact**
- kernel_size가 stride로 나누어떨어지지 않으면 격자 패턴 발생
- 해결: kernel=4, stride=2 (4÷2=2 ✓)

</details>

## 문제 3: Embedding Layer의 학습 과정

**문제:** CGAN에서 Embedding Layer가 학습되는 과정을 설명하고, 왜 의미있는 벡터 표현을 학습할 수 있는지 서술하세요.

---

### 여러분의 답안:
```
[여기에 답안을 작성하세요]


```

<details>
<summary><b>모범 답안 보기</b></summary>

**Embedding Layer의 학습 과정:**

**초기 상태 (랜덤 초기화):**
```python
embedding = nn.Embedding(num_classes=10, embedding_dim=50)
# 가중치 shape: (10, 50)
# 초기값: N(0, 0.02) 정규분포

Class 0: [0.01, -0.02, 0.03, ..., -0.01]  (랜덤)
Class 7: [-0.01, 0.02, -0.03, ..., 0.02]  (랜덤)
→ 초기에는 의미 없음
```

**학습 과정:**
```
1단계: Generator 학습
noise + embedding[7] → Sneaker 이미지 생성
→ D가 "진짜"라고 판단하면 G의 loss 감소
→ Backpropagation으로 embedding[7]도 업데이트!

2단계: Discriminator 학습
Sneaker 이미지 + embedding[7] → "진짜" 판별
T-shirt 이미지 + embedding[7] → "가짜" 판별 (페어링 안 맞음)
→ D가 "올바른 페어링"을 학습
→ 이 과정에서 embedding이 각 클래스의 특징을 학습

3단계: 반복
수천 번의 반복 후, embedding 벡터는:
- 같은 카테고리 클래스끼리 유사해짐
- 다른 카테고리는 멀어짐
```

**왜 의미있는 벡터가 되는가?**

1. **Gradient 압력:**
   - G는 "이 레이블에 맞는" 이미지를 만들어야 함
   - D는 "이미지와 레이블이 맞는지" 판단해야 함
   - Embedding은 이 두 압력 사이에서 최적화됨

2. **유사한 클래스 → 유사한 특징:**
   ```
   T-shirt (0)과 Shirt (6):
   → 둘 다 "상의"라는 공통점
   → Generator가 비슷한 패턴을 생성해야 함
   → Embedding도 비슷한 방향으로 학습됨
   ```

3. **다른 클래스 → 다른 특징:**
   ```
   T-shirt (0)과 Sneaker (7):
   → 완전히 다른 형태
   → Discriminator가 확실히 구분해야 함
   → Embedding이 멀어지는 방향으로 학습됨
   ```

**학습 후 결과 (예시):**
```python
# 임베딩 코사인 유사도
             T-shirt  Shirt  Sneaker  Trouser
T-shirt       1.00    0.85    0.32     0.41
Shirt         0.85    1.00    0.28     0.39
Sneaker       0.32    0.28    1.00     0.35
Trouser       0.41    0.39    0.35     1.00

관찰:
- T-shirt와 Shirt: 높은 유사도 (0.85) - 둘 다 상의!
- T-shirt와 Sneaker: 낮은 유사도 (0.32) - 완전히 다른 품목
```

**결론:**
- Embedding은 단순한 인덱스가 아닌, **학습된 의미 벡터**
- GAN의 학습 과정에서 자연스럽게 의미 구조를 포착
- Word2Vec, BERT 등의 임베딩과 유사한 원리

</details>

---
# Part 3: Shape 추적 문제 📐

## 문제 1: DCGAN Generator Shape 추적

**문제:** DCGAN Generator에서 노이즈 벡터가 64×64×3 이미지가 되기까지의 shape 변화를 채우세요.

**입력:**
- batch_size = 4
- noise_dim = 100
- 목표 이미지 크기 = 64×64×3 (RGB)

**각 단계의 shape을 채우세요:**

1. 입력 노이즈: (4, 100)
2. Dense 후: (4, _____)  # 4×4×512로 만들 예정
3. Reshape 후: (4, 512, _____, _____)
4. ConvTranspose 1 (stride=2): (4, 256, _____, _____)
5. ConvTranspose 2 (stride=2): (4, 128, _____, _____)
6. ConvTranspose 3 (stride=2): (4, 64, _____, _____)
7. ConvTranspose 4 (stride=2, 출력층): (4, _____, _____, _____)

<details>
<summary><b>정답 보기</b></summary>

1. 입력 노이즈: **(4, 100)**
2. Dense 후: **(4, 8192)**  # 4×4×512 = 8192
3. Reshape 후: **(4, 512, 4, 4)**
4. ConvTranspose 1: **(4, 256, 8, 8)**
5. ConvTranspose 2: **(4, 128, 16, 16)**
6. ConvTranspose 3: **(4, 64, 32, 32)**
7. ConvTranspose 4: **(4, 3, 64, 64)**  # RGB 3채널

**PyTorch 코드:**
```python
class DCGANGenerator(nn.Module):
    def __init__(self, noise_dim=100):
        super().__init__()
        
        # Dense + Reshape
        self.fc = nn.Linear(noise_dim, 4*4*512)
        
        self.main = nn.Sequential(
            # 4×4 → 8×8
            nn.ConvTranspose2d(512, 256, 4, 2, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            
            # 8×8 → 16×16
            nn.ConvTranspose2d(256, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            
            # 16×16 → 32×32
            nn.ConvTranspose2d(128, 64, 4, 2, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            
            # 32×32 → 64×64
            nn.ConvTranspose2d(64, 3, 4, 2, 1),
            nn.Tanh()
        )
    
    def forward(self, z):
        x = self.fc(z).view(-1, 512, 4, 4)
        return self.main(x)
```

</details>

## 문제 2: CGAN Shape 추적

**문제:** CGAN Generator에서 노이즈 + 레이블이 28×28×1 이미지가 되기까지의 shape을 채우세요.

**입력:**
- batch_size = 8
- noise_dim = 100
- num_classes = 10
- embedding_dim = 50
- 목표 이미지 크기 = 28×28×1 (Grayscale)

**각 단계의 shape을 채우세요:**

1. 입력 노이즈: (8, 100)
2. 입력 레이블: (8,)  # 정수
3. 레이블 임베딩 후: (8, _____)
4. Concatenate [noise, embedding]: (8, _____)
5. Dense 후: (8, 12544)  # 7×7×256
6. Reshape 후: (8, 256, _____, _____)
7. ConvTranspose 1: (8, 128, _____, _____)
8. ConvTranspose 2 (출력층): (8, _____, _____, _____)

<details>
<summary><b>정답 보기</b></summary>

1. 입력 노이즈: **(8, 100)**
2. 입력 레이블: **(8,)**  # LongTensor
3. 레이블 임베딩 후: **(8, 50)**
4. Concatenate: **(8, 150)**  # 100 + 50
5. Dense 후: **(8, 12544)**
6. Reshape 후: **(8, 256, 7, 7)**
7. ConvTranspose 1: **(8, 128, 14, 14)**
8. ConvTranspose 2: **(8, 1, 28, 28)**  # Grayscale

**PyTorch 코드:**
```python
class ConditionalGenerator(nn.Module):
    def __init__(self, noise_dim=100, num_classes=10, embedding_dim=50):
        super().__init__()
        
        # 레이블 임베딩
        self.label_embedding = nn.Embedding(num_classes, embedding_dim)
        
        # Dense
        self.fc = nn.Sequential(
            nn.Linear(noise_dim + embedding_dim, 7*7*256),
            nn.BatchNorm1d(7*7*256),
            nn.ReLU(True)
        )
        
        # ConvTranspose
        self.main = nn.Sequential(
            # 7×7 → 14×14
            nn.ConvTranspose2d(256, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            
            # 14×14 → 28×28
            nn.ConvTranspose2d(128, 1, 4, 2, 1),
            nn.Tanh()
        )
    
    def forward(self, z, labels):
        # (8, 100) + (8, 50) → (8, 150)
        label_embed = self.label_embedding(labels)
        gen_input = torch.cat([z, label_embed], dim=1)
        
        # (8, 150) → (8, 12544) → (8, 256, 7, 7)
        x = self.fc(gen_input).view(-1, 256, 7, 7)
        
        # (8, 256, 7, 7) → (8, 1, 28, 28)
        return self.main(x)
```

</details>

---
# Part 4: 코드 구현 문제 (PyTorch) 💻

## 문제 1: DCGAN Generator 구현

**문제:** 빈칸을 채워 DCGAN Generator를 완성하세요. (MNIST 28×28 기준)

In [ ]:
# TODO: DCGAN Generator 완성
class DCGANGenerator(nn.Module):
    def __init__(self, noise_dim=100):
        super(DCGANGenerator, self).__init__()
        
        # Dense + BatchNorm + ReLU
        self.fc = nn.Sequential(
            nn.Linear(noise_dim, 7*7*256),
            # TODO: BatchNorm1d 추가
            # 코드:
            nn.ReLU(True)
        )
        
        self.main = nn.Sequential(
            # 7×7 → 14×14
            nn.ConvTranspose2d(256, 128, 4, stride=2, padding=1),
            # TODO: BatchNorm2d 추가
            # 코드:
            nn.ReLU(True),
            
            # 14×14 → 28×28 (출력층)
            nn.ConvTranspose2d(128, 1, 4, stride=2, padding=1),
            # TODO: 출력 활성화 함수는?
            # 코드:
        )
    
    def forward(self, z):
        x = self.fc(z)
        x = x.view(-1, 256, 7, 7)  # Reshape
        return self.main(x)

<details>
<summary><b>모범 답안 보기</b></summary>

```python
# Dense 후 BatchNorm1d
nn.BatchNorm1d(7*7*256),

# ConvTranspose 후 BatchNorm2d
nn.BatchNorm2d(128),

# 출력 활성화 함수
nn.Tanh()
```

**핵심 포인트:**
- BatchNorm1d: 1D 텐서 (batch, features)
- BatchNorm2d: 4D 텐서 (batch, channels, height, width)
- 출력층에는 BatchNorm 없음!

</details>

## 문제 2: DCGAN Discriminator 구현

**문제:** 빈칸을 채워 DCGAN Discriminator를 완성하세요.

In [ ]:
# TODO: DCGAN Discriminator 완성
class DCGANDiscriminator(nn.Module):
    def __init__(self):
        super(DCGANDiscriminator, self).__init__()
        
        self.main = nn.Sequential(
            # 28×28 → 14×14 (입력층 - BatchNorm 없음!)
            nn.Conv2d(1, 64, 4, stride=2, padding=1),
            # TODO: 활성화 함수는? (LeakyReLU 사용)
            # 코드:
            
            # 14×14 → 7×7
            nn.Conv2d(64, 128, 4, stride=2, padding=1),
            # TODO: BatchNorm2d 추가
            # 코드:
            # TODO: 활성화 함수 (동일)
            # 코드:
            
            # Flatten + Dense
            nn.Flatten(),
            nn.Linear(128*7*7, 1),
            # TODO: 출력 활성화 함수는? (Sigmoid 또는 없음)
            # 코드:
        )
    
    def forward(self, x):
        return self.main(x)

<details>
<summary><b>모범 답안 보기</b></summary>

```python
# 활성화 함수 (LeakyReLU)
nn.LeakyReLU(0.2, inplace=True),

# BatchNorm2d
nn.BatchNorm2d(128),

# 출력 활성화 함수
nn.Sigmoid()  # 또는 없음 (BCEWithLogitsLoss 사용 시)
```

**핵심 포인트:**
- 입력층에는 BatchNorm 없음!
- LeakyReLU(0.2): alpha=0.2 (DCGAN 표준)
- Sigmoid vs BCEWithLogitsLoss: 둘 중 하나 선택

</details>

## 문제 3: Conditional Generator 구현

**문제:** Embedding을 사용하는 CGAN Generator를 완성하세요.

In [ ]:
# TODO: Conditional Generator 완성
class ConditionalGenerator(nn.Module):
    def __init__(self, noise_dim=100, num_classes=10, embedding_dim=50):
        super(ConditionalGenerator, self).__init__()
        
        # TODO: 레이블 임베딩 레이어 정의
        # 힌트: nn.Embedding(num_embeddings, embedding_dim)
        # 코드:
        self.label_embedding = 
        
        # Dense
        input_dim = noise_dim + embedding_dim
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 7*7*256),
            nn.BatchNorm1d(7*7*256),
            nn.ReLU(True)
        )
        
        self.main = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(128, 1, 4, 2, 1),
            nn.Tanh()
        )
    
    def forward(self, z, labels):
        # TODO: 레이블을 임베딩하고 노이즈와 결합
        # 힌트: torch.cat([z, embedding], dim=1)
        # 코드:
        label_embed = 
        gen_input = 
        
        x = self.fc(gen_input)
        x = x.view(-1, 256, 7, 7)
        return self.main(x)

<details>
<summary><b>모범 답안 보기</b></summary>

```python
# 레이블 임베딩
self.label_embedding = nn.Embedding(num_classes, embedding_dim)

# forward 함수
label_embed = self.label_embedding(labels)  # (batch, embedding_dim)
gen_input = torch.cat([z, label_embed], dim=1)  # (batch, noise_dim + embedding_dim)
```

**핵심 포인트:**
- Embedding Layer: 정수 인덱스 → 학습 가능한 벡터
- torch.cat: dim=1로 feature 차원에서 결합
- 노이즈와 임베딩을 합쳐서 Generator 입력으로 사용

</details>

## 문제 4: Embedding vs One-Hot Encoding 비교 실험

**문제:** 두 방식의 파라미터 수와 의미 학습 능력을 비교하는 코드를 작성하세요.

In [ ]:
# TODO: Embedding vs One-Hot 비교
num_classes = 10
embedding_dim = 50

# 방식 1: Embedding
embedding_layer = nn.Embedding(num_classes, embedding_dim)

# TODO: Embedding Layer의 파라미터 수는?
# 힌트: num_classes * embedding_dim
embedding_params = 

print(f"Embedding 방식 파라미터 수: {embedding_params}")

# 방식 2: One-Hot (Linear layer로 구현)
# One-Hot을 입력받아 변환하는 Linear layer
onehot_layer = nn.Linear(num_classes, embedding_dim)

# TODO: One-Hot 방식의 파라미터 수는?
# 힌트: num_classes * embedding_dim + bias
onehot_params = 

print(f"One-Hot 방식 파라미터 수: {onehot_params}")

# TODO: 1000개 클래스일 때는?
print(f"\n1000개 클래스 (embedding_dim=100):")
print(f"Embedding: {1000 * 100:,}개")
print(f"One-Hot 입력 크기: {1000}차원")

<details>
<summary><b>모범 답안 보기</b></summary>

```python
# Embedding 파라미터 수
embedding_params = num_classes * embedding_dim  # 10 * 50 = 500

# One-Hot 방식 파라미터 수
onehot_params = num_classes * embedding_dim + embedding_dim  # 10*50 + 50 = 550 (bias 포함)
```

**비교 분석:**

| 클래스 수 | Embedding 파라미터 | One-Hot 입력 크기 | 의미 학습 |
|----------|-------------------|------------------|----------|
| **10** | 500 (10×50) | 10차원 | ✅ 가능 |
| **100** | 5,000 (100×50) | 100차원 | ✅ 가능 |
| **1,000** | 50,000 (1000×50) | 1,000차원 | ✅ 가능 |
| **10,000** | 500,000 (10000×50) | 10,000차원 | ✅ 가능 |

**One-Hot의 문제:**
- 클래스 수가 많아지면 입력 차원이 폭발적으로 증가
- 모든 클래스가 직교 벡터 → 유사도 학습 불가
- 메모리 비효율 (대부분 0, 하나만 1)

**Embedding의 장점:**
- 차원을 자유롭게 조절 가능 (50, 100, 200 등)
- 유사한 클래스는 유사한 벡터로 학습됨
- 파라미터 효율적

**실험 코드:**
```python
# 학습 후 임베딩 유사도 확인
with torch.no_grad():
    embeddings = embedding_layer.weight  # (10, 50)
    
    # 코사인 유사도 계산
    from sklearn.metrics.pairwise import cosine_similarity
    similarity = cosine_similarity(embeddings.cpu().numpy())
    
    # 시각화
    import seaborn as sns
    plt.figure(figsize=(8, 6))
    sns.heatmap(similarity, annot=True, fmt='.2f', cmap='coolwarm',
                xticklabels=range(10), yticklabels=range(10))
    plt.title('클래스 임베딩 유사도')
    plt.show()
```

</details>

---
# Part 5: 미니 프로젝트 - Fashion MNIST CGAN 🎨

## 프로젝트: Fashion MNIST로 조건부 이미지 생성

**목표:** 원하는 옷 종류(T-shirt, Sneaker 등)를 지정하여 생성하는 CGAN을 완성합니다.

In [ ]:
# 1. 데이터 로드
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

train_dataset = datasets.FashionMNIST(
    root='./data',
    train=True,
    transform=transform,
    download=True
)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=128,
    shuffle=True,
    drop_last=True
)

# 클래스 이름
CLASS_NAMES = [
    "T-shirt", "Trouser", "Pullover", "Dress", "Coat",
    "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"
]

print(f"✅ 데이터셋 로드 완료: {len(train_dataset)}장")
print(f"클래스: {CLASS_NAMES}")

In [ ]:
# 2. 모델 정의 (위에서 완성한 ConditionalGenerator 사용)
G = ConditionalGenerator(
    noise_dim=100,
    num_classes=10,
    embedding_dim=50
).to(device)

class ConditionalDiscriminator(nn.Module):
    def __init__(self, num_classes=10, embedding_dim=50):
        super().__init__()
        self.label_embedding = nn.Embedding(num_classes, embedding_dim)
        self.label_fc = nn.Linear(embedding_dim, 28*28)
        
        self.main = nn.Sequential(
            # 입력: (batch, 2, 28, 28) - 이미지(1) + 레이블맵(1)
            nn.Conv2d(2, 64, 4, 2, 1),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(64, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Flatten(),
            nn.Linear(128*7*7, 1),
            nn.Sigmoid()
        )
    
    def forward(self, img, labels):
        # 레이블을 28×28 맵으로 변환
        label_embed = self.label_embedding(labels)
        label_map = self.label_fc(label_embed).view(-1, 1, 28, 28)
        
        # 이미지와 결합
        disc_input = torch.cat([img, label_map], dim=1)
        return self.main(disc_input)

D = ConditionalDiscriminator().to(device)

# Optimizer
optimizer_G = optim.Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = optim.Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))

criterion = nn.BCELoss()

print("✅ 모델 초기화 완료")
print(f"Generator 파라미터: {sum(p.numel() for p in G.parameters()):,}개")
print(f"Discriminator 파라미터: {sum(p.numel() for p in D.parameters()):,}개")

In [ ]:
# 3. 학습 함수
def train_step_cgan(real_images, real_labels):
    """CGAN 한 스텝 학습"""
    batch_size = real_images.size(0)
    real_images = real_images.to(device)
    real_labels = real_labels.to(device)
    
    # 타겟 레이블
    valid = torch.ones(batch_size, 1, device=device)
    fake = torch.zeros(batch_size, 1, device=device)
    
    # ========================================
    # Discriminator 학습
    # ========================================
    optimizer_D.zero_grad()
    
    # 진짜 이미지 판별
    real_output = D(real_images, real_labels)
    d_loss_real = criterion(real_output, valid)
    
    # 가짜 이미지 생성 및 판별
    noise = torch.randn(batch_size, 100, device=device)
    fake_labels = torch.randint(0, 10, (batch_size,), device=device)
    fake_images = G(noise, fake_labels)
    
    fake_output = D(fake_images.detach(), fake_labels)
    d_loss_fake = criterion(fake_output, fake)
    
    # D Loss
    d_loss = d_loss_real + d_loss_fake
    d_loss.backward()
    optimizer_D.step()
    
    # ========================================
    # Generator 학습
    # ========================================
    optimizer_G.zero_grad()
    
    noise = torch.randn(batch_size, 100, device=device)
    fake_labels = torch.randint(0, 10, (batch_size,), device=device)
    fake_images = G(noise, fake_labels)
    
    fake_output = D(fake_images, fake_labels)
    g_loss = criterion(fake_output, valid)
    
    g_loss.backward()
    optimizer_G.step()
    
    return d_loss.item(), g_loss.item()

In [ ]:
# 4. 학습 실행 (빠른 데모용 5 에폭)
print("🚀 CGAN 학습 시작 (5 에폭)")

# 고정 노이즈 (각 클래스별 1개씩)
fixed_noise = torch.randn(10, 100, device=device)
fixed_labels = torch.arange(0, 10, device=device)

num_epochs = 5
for epoch in range(1, num_epochs + 1):
    epoch_d_loss = []
    epoch_g_loss = []
    
    for batch_idx, (images, labels) in enumerate(train_dataloader):
        d_loss, g_loss = train_step_cgan(images, labels)
        epoch_d_loss.append(d_loss)
        epoch_g_loss.append(g_loss)
    
    avg_d = np.mean(epoch_d_loss)
    avg_g = np.mean(epoch_g_loss)
    
    print(f"Epoch [{epoch}/{num_epochs}] D Loss: {avg_d:.4f}, G Loss: {avg_g:.4f}")
    
    # 결과 시각화
    with torch.no_grad():
        fake_images = G(fixed_noise, fixed_labels).cpu()
        fake_images = (fake_images + 1) / 2  # [-1,1] → [0,1]
        
        fig, axes = plt.subplots(2, 5, figsize=(12, 5))
        for i, ax in enumerate(axes.flat):
            ax.imshow(fake_images[i].squeeze(), cmap='gray')
            ax.set_title(CLASS_NAMES[i], fontsize=10)
            ax.axis('off')
        
        plt.suptitle(f'Epoch {epoch} - 각 클래스별 생성 결과', fontsize=14)
        plt.tight_layout()
        plt.show()

print("🎉 학습 완료!")

In [ ]:
# 5. 특정 클래스 여러 개 생성
def generate_specific_class(class_idx, num_samples=16):
    """특정 클래스 이미지 여러 개 생성"""
    G.eval()
    
    noise = torch.randn(num_samples, 100, device=device)
    labels = torch.full((num_samples,), class_idx, device=device)
    
    with torch.no_grad():
        images = G(noise, labels).cpu()
        images = (images + 1) / 2
    
    fig, axes = plt.subplots(4, 4, figsize=(10, 10))
    for i, ax in enumerate(axes.flat):
        ax.imshow(images[i].squeeze(), cmap='gray')
        ax.axis('off')
    
    plt.suptitle(f'{CLASS_NAMES[class_idx]} 16개 생성', fontsize=16)
    plt.tight_layout()
    plt.show()
    
    G.train()

# 스니커즈 16개 생성
print("👟 스니커즈 16개 생성:")
generate_specific_class(class_idx=7, num_samples=16)

# 드레스 16개 생성
print("👗 드레스 16개 생성:")
generate_specific_class(class_idx=3, num_samples=16)

---
## 📝 요약 및 다음 단계

**오늘 복습한 핵심:**

### DCGAN
1. **5가지 설계 원칙**: Strided Conv, FC 제거, BatchNorm, 적절한 활성화, Adam
2. **Transposed Convolution**: 학습 가능한 업샘플링
3. **아키텍처**: Encoder-Decoder 구조, ResNet 블록

### CGAN
1. **조건 정보**: Generator와 Discriminator 모두 조건 입력
2. **Embedding**: 학습 가능한 의미 벡터 (vs One-Hot)
3. **활용**: 원하는 클래스 지정 생성

**핵심 차이점:**

| 항목 | Vanilla GAN | DCGAN | CGAN |
|------|------------|-------|------|
| **구조** | FC Layer | Convolution | Convolution + Condition |
| **제어** | 불가능 | 불가능 | 가능 (레이블 지정) |
| **품질** | 낮음 | 높음 | 높음 |
| **해상도** | 28×28 | 64×64+ | 28×28+ |

**다음 단계:**
- **CycleGAN**: Unpaired Image-to-Image Translation (말 ↔ 얼룩말)
- **StyleGAN**: 고품질 얼굴 생성
- **Diffusion Models**: 최신 생성 모델

**실전 팁:**
1. DCGAN 원칙은 거의 모든 GAN의 기초
2. Embedding은 CGAN뿐 아니라 다양한 곳에 활용
3. Shape 추적은 디버깅의 핵심

수고하셨습니다! 🎉